# 0. Correlation Analysis

This notebook performs an $f$-$k$ analysis and filtering, as well as phase velocity dispersion estimation.

# 0. Packages and setup

Import the necessary Python packages and add a few lines to embellish figures.

In [ ]:
import obspy
import numpy as np
from pandas import read_csv
from pandas import read_csv
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from scipy.io.wavfile import write
from obspy.signal.filter import bandpass

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 70})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

# 1. General input

Basic input, including the file to be read and the average spacing between channels.

In [ ]:
# Input file.
input_file='/Users/andreas/Desktop/EruptionAnalysis/eruption.npy'
# Scale for plotting.
scale=5
# Receiver spacing [m].
dx=8.0
# Sampling rate [Hz].
dt=1.0/200.0

# 2. Data reading and plotting

## 2.1. Read and plot complete data

In [ ]:
cct=np.load(input_file)

In [ ]:
print(cct.shape)

nt=cct.shape[1]
nx=cct.shape[0]-1

t=np.linspace(0.0,nt*dt,nt)

In [ ]:
plt.figure(figsize=(30,60))

for i,j in enumerate(cct):
    
    if i % 10 == 0: print(f"At {i} of {np.shape(cct)[0]}")
    
    data = j
    data /= np.max(np.abs(data))        
    dist_var = (i-1)*dx
    
    plt.plot(t,(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t,(scale*data)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.grid()
plt.savefig('OUTPUT/raw.png',dpi=250)
plt.show()

## 2.1.1. Select a subset of the data for further analysis

In [ ]:
cct=cct[320:,:30000]
#cct=cct[450:,:]
print(np.shape(cct))

nx=cct.shape[0]-1
nt=cct.shape[1]

t=np.linspace(0.0,nt*dt,nt)

In [ ]:
plt.figure(figsize=(30,60))

for i,j in enumerate(cct):
    
    if i % 10 == 0: print(f"At {i} of {np.shape(cct)[0]}")
    
    data = j
    data /= np.max(np.abs(data))        
    dist_var = (i-1)*dx
    
    plt.plot(t,(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t,(scale*data)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.grid()
plt.savefig('OUTPUT/raw_cropped.png',dpi=250)
plt.show()

## 2.2. Perform some preprocessing

### 2.2.2. Time-domain filtering, downsampling and tapering

In [ ]:
# Minimum and maximum frequencies [Hz].
freqmin=0.01
freqmax=10.0
# Downsampling factor.
downsample=1

In [ ]:
# Frequency-domain filtering.
cct_filt=np.zeros(np.shape(cct))
for i in range(cct.shape[0]-1): cct_filt[i,:]=bandpass(cct[i,:],freqmin=freqmin,freqmax=freqmax,df=1.0/dt,corners=4,zerophase=True)

In [ ]:
# Downsampling.
cct_filt_down=cct_filt[:,0:nt:downsample]

nx=cct_filt_down.shape[0]-1
nt=cct_filt_down.shape[1]
dt=downsample*dt

t=np.linspace(0.0,nt*dt,nt)

In [ ]:
cct_filt_down_tap=cct_filt_down.copy()

# Taper in space domain.
width=3
for i in range(width): 
    cct_filt_down_tap[i,:]=(np.float(i+1)/np.float(width+1))*cct_filt_down[i,:]
    cct_filt_down_tap[nx-i,:]=(np.float(i+1)/np.float(width+1))*cct_filt_down[nx-i,:]

In [ ]:
plt.figure(figsize=(30,60))
scale = 10.0
norm = np.max(np.abs(cct_filt_down))

for i in range(nx):

    data = cct_filt_down_tap[i,:]
    dist_var = (i-1)*dx
    
    plt.plot(t,(scale*data/norm)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t,(scale*data/norm)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data/norm+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.grid()
filename='OUTPUT/f-filtered_'+str(freqmin)+'-'+str(freqmax)+'Hz.png'
plt.savefig(filename,dpi=250)
plt.show()

### 2.2.3. Turn selected traces into audio files

# 3. Frequency-wavenumber domain

Before any $f$-$k$ filtering, we transform the time-domain data into the frequency-wavenumber domain.

## 3.1. Compute and plot frequency-wavenumber domain representation

We apply a 2D Fast Fourier Transform and the roll the indices in order to bring it into the physical frequency domain.

In [ ]:
# 2D Fourier transform.
ccf=np.fft.fft2(cct_filt_down_tap)
# Roll in order to plot in physical f-k domain.
ccfr=np.roll(np.roll(ccf,int((nt-1)/2),axis=1),int((nx-1)/2),axis=0)
# Make frequency and wavenumber axes.
f=np.linspace(-0.5/dt,0.5/dt,nt)
k=np.linspace(-np.pi/dx,np.pi/dx,nx+1)
ff,kk=np.meshgrid(f,k)

Make a smoothed version of the power spectrum for plotting.

# 4. $f$-$k$ filtering

For $f$-$k$ filtering we define a mask that cuts out part of the $f$-$k$ spectrum.

## 4.1. Input

Set parameters that define the frequency-wavenumber mask.

In [ ]:
# Minimum and maximum absolute phase velocities [m/s].
c_min=250.0
c_max=1000.0

## 4.2. $f$-$k$ mask

Build the $f$-$k$ mask for both the forward- and backward-propagating (reflected) part of the correlation wavefield.

In [ ]:
# Mask for forward-propagating wavefield.
mask_fwd=np.ones(np.shape(ccfr),dtype='complex64')
# Mask for backward-propagating (reflected) wavefield.
mask_rev=np.ones(np.shape(ccfr),dtype='complex64')

# Apply the mask.
for i in range(nx):
    for j in range(nt):
        
        # Compute phase velocity.
        if np.abs(k[i]):
            c=2.0*np.pi*f[j]/k[i]
        else:
            c=1.0e9
            
        # Maximum and minimum absolute phase velocities.
        if (np.abs(c)>c_max) or (np.abs(c)<c_min):
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
        # Forward or backward propagation.
        if c>0.0: mask_fwd[i,j]=0.0
        if c<0.0: mask_rev[i,j]=0.0
        # Bandpass filter (more generous because already filtered before).
        if np.abs(f[j])>1.2*freqmax: 
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
        if np.abs(f[j])<0.8*freqmin: 
            mask_fwd[i,j]=0.0
            mask_rev[i,j]=0.0
            
            
# Smooth the mask.
dummy_fwd=mask_fwd.copy()
mask_fwd_smooth=mask_fwd.copy()
dummy_rev=mask_rev.copy()
mask_rev_smooth=mask_rev.copy()

for l in range(10):
    for i in np.arange(1,nx-1):
        mask_rev_smooth[i,:]=(dummy_rev[i,:]+dummy_rev[i-1,:]+dummy_rev[i+1,:])/3.0
        mask_fwd_smooth[i,:]=(dummy_fwd[i,:]+dummy_fwd[i-1,:]+dummy_fwd[i+1,:])/3.0
    dummy_rev=mask_rev_smooth
    dummy_fwd=mask_fwd_smooth

for l in range(10):
    for j in np.arange(1,nt-1):
        mask_rev_smooth[:,j]=(dummy_rev[:,j]+dummy_rev[:,j-1]+dummy_rev[:,j+1])/3.0
        mask_fwd_smooth[:,j]=(dummy_fwd[:,j]+dummy_fwd[:,j-1]+dummy_fwd[:,j+1])/3.0
    dummy_rev=mask_rev_smooth
    dummy_fwd=mask_fwd_smooth

In [ ]:
# Apply the f-k domain filter.
ccfr_fwd_filtered=ccfr*mask_fwd_smooth
ccfr_rev_filtered=ccfr*mask_rev_smooth

## 4.3. Return to the time domain

After application of the mask (filter) we roll the arrays back and then apply the 2D inverse transform.

In [ ]:
# Roll back.
ccf_fwd_filtered=np.roll(np.roll(ccfr_fwd_filtered,-int((nt-1)/2),axis=1),-int((nx-1)/2),axis=0)
ccf_rev_filtered=np.roll(np.roll(ccfr_rev_filtered,-int((nt-1)/2),axis=1),-int((nx-1)/2),axis=0)

# Inverse 2D Fourier transform.
cct_fwd_filtered=np.real(np.fft.ifft2(ccf_fwd_filtered))
cct_rev_filtered=np.real(np.fft.ifft2(ccf_rev_filtered))

In [ ]:
# Plotting options.
# Wavefield propagating away from the interrogator towards the eruption site.
plot_forward=False
# Wavefield propagating from the eruption site towards the interrogator.
plot_reverse=True

# Plot filtered version.
plt.figure(figsize=(30,60))

scale=30
norm=np.max(np.abs(cct_filt_down)) 

for i in range(nx):
    
    dist_var=(i-1)*dx 
    
    if plot_forward:
        data_fwd=cct_fwd_filtered[i,:]
        plt.plot(t,(scale*data_fwd/norm)+dist_var,'k-', alpha = 0.4) 
        plt.fill_between(t,(scale*data_fwd/norm)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data_fwd/norm+dist_var>=dist_var), interpolate=True,fc='r',alpha=0.8)
        
    if plot_reverse:
        data_rev=cct_rev_filtered[i,:]
        plt.plot(t,(scale*data_rev/norm)+dist_var,'k-', alpha = 0.4) 
        plt.fill_between(t,(scale*data_rev/norm)+dist_var,y2=np.ones(np.shape(t))*dist_var,where=(data_rev/norm+dist_var>=dist_var), interpolate=True,fc='b',alpha=0.8)

plt.xlabel("time [s]")
plt.ylabel("distance [m]")
plt.grid()
#plt.xlim([0.0,200.0])
#plt.ylim([0.0,580.0])
filename='OUTPUT/fk-filtered_'+str(freqmin)+'-'+str(freqmax)+'Hz-'+str(c_min)+'-'+str(c_max)+'mps.png'
plt.savefig(filename,dpi=250)
plt.show()